In [1]:
# Construct score volume from score maps of each landmark

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

import time

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
from multiprocess import Pool

Setting environment for Gordon


In [3]:
from conversion import images_to_volume

In [4]:
train_sample_scheme = 1

In [5]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

In [6]:
downscale_factor = 32
voxel_z_size = section_thickness/(xy_pixel_distance_lossless * downscale_factor)

In [13]:
def load_scoremap_worker(sec, stack, label, downscale_factor):
    try:
#         scoremap = DataManager.load_scoremap(stack=stack, section=sec, label=label, downscale_factor=downscale_factor)
        scoremap = DataManager.load_scoremap(stack=stack, section=sec, label=label, 
                                             downscale_factor=downscale_factor, 
                                             train_sample_scheme=train_sample_scheme)
        return (sec-1, scoremap)
    except Exception as e:
        pass
    
f = lambda sec: load_scoremap_worker(sec, stack, label, downscale_factor)
    
def load_scoremaps_parallel(sections, stack, label, downscale_factor):
    pool = Pool(4)
    index_scoremap_tuples = pool.map(lambda sec: load_scoremap_worker(sec, stack, label, downscale_factor), 
                                     sections)
    return dict(filter(None, index_scoremap_tuples))

def load_scoremaps_sequential(sections, stack, label, downscale_factor):
    scoremaps = {}
    for sec in sections:
        t = time.time()
        try:
            scoremaps[sec-1] = DataManager.load_scoremap(stack=stack, section=sec, label=label, downscale_factor=downscale_factor)
        except:
            pass
        print time.time() - t
    return scoremaps

In [14]:
for stack in ['MD591']:
    
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    
#     for label in structures:
    for label in ['7N']:
#     for label in ['Amb']:

        print label
    
        t = time.time()
        scoremaps = load_scoremaps_parallel(stack=stack, sections=range(first_sec, last_sec+1), label=label, downscale_factor=downscale_factor)
#         scoremaps = load_scoremaps_sequential(stack=stack, sections=range(first_sec, last_sec+1), label=label, downscale_factor=downscale_factor)
#         scoremaps = load_scoremaps_parallel(stack=stack, sections=range(200, 300), label=label, downscale_factor=downscale_factor)
#         scoremaps = load_scoremaps_sequential(stack=stack, sections=range(200, 220), label=label, downscale_factor=downscale_factor)
        sys.stderr.write('Load scoremaps: %.2f seconds\n' % (time.time() - t))
        
#         del scoremaps
        
        score_volume, score_volume_bbox = images_to_volume(images=scoremaps, voxel_size=(1, 1, voxel_z_size), first_sec=first_sec-1, last_sec=last_sec-1)
        
        output_dir = create_if_not_exists(os.path.join(VOLUME_ROOTDIR, stack, 'score_volumes'))
#         score_volume_filepath = os.path.join(output_dir, '%(stack)s_down%(ds)d_scoreVolume_%(label)s.bp' % \
#                                             dict(stack=stack, ds=downscale_factor, label=label))

        score_volume_filepath = DataManager.get_score_volume_filepath(stack=stack, downscale=downscale_factor, label=label, train_sample_scheme=train_sample_scheme)
        bp.pack_ndarray_file(score_volume.astype(np.float16), score_volume_filepath)
        
        sys.stderr.write('Create score volume: %.2f seconds\n' % (time.time() - t)) # 50 seconds - load scoremap 4 processes

        score_volume_bbox_filepath = DataManager.get_score_volume_bbox_filepath(stack=stack, downscale=downscale_factor, label=label)
        np.savetxt(score_volume_bbox_filepath, np.array(score_volume_bbox)[None])
    
        del score_volume, scoremaps

7N


Load scoremaps: 77.73 seconds
Create score volume: 80.55 seconds


In [ ]:
# Read

In [11]:
score_volume = DataManager.load_score_volume(stack='MD603', label='7N', downscale=32, train_sample_scheme=1)

In [12]:
score_volume.dtype

dtype('float32')

In [ ]:
# plt.imshow(score_volume[:,:,100], cmap=plt.cm.hot)

In [ ]:
# volume_roi = bp.unpack_ndarray_file(volume_dir + '/%(stack)s_scoreVolume_%(label)s.bp' % {'stack': stack, 'label': '5N'})
# plt.imshow(volume_roi[..., 300]);

In [ ]:
# from skimage.transform import rescale, resize

# def load_scoremap_tb(sec):
#     ''' 
#     Get scoremap. 
#     Assume scoremap is stored as a 2D array for scores of the full frame at thumbnail scale.
    
#     Parameters
#     ----------
#     sec : int
#         section number
    
#     Returns
#     -------
#     2d array
#         downsampled full-frame scoremap
#     '''
#     global stack
        
#     img_fn = contourMap_rootdir + '/%(stack)s/%(sec)04d/%(stack)s_%(sec)04d_denseScoremapThumbnail_outerContour.hdf' % \
#                 {'stack': stack, 'sec': sec}

#     contour_prob_map = load_hdf(img_fn)
    
# #     # crop at upper border of ROI box
# #     xmin, ymin, w, h = detect_bbox_lookup[stack]
# #     contour_prob_map[:ymin] = 0

#     scoremap = resize(contour_prob_map, (h_down, w_down))
#     return scoremap